In [34]:
import pandas as pd
import geopandas as gpd
import mercantile
from tqdm import tqdm
import tempfile
import os
from shapely import geometry
import time
import json

In [35]:
aoi_geom = {
    "coordinates": [
        [
            [-76.15741548689954, 43.05692144640927], 
            [-76.15741548689954, 43.05635088078997],  
            [-76.15648427005196, 43.05635088078997],  
            [-76.15648427005196, 43.05692144640927],  
            [-76.15741548689954, 43.05692144640927], 
        ]
    ],
    "type": "Polygon"
}
aoi_shape = geometry.shape(aoi_geom)

minx, miny, maxx, maxy = aoi_shape.bounds


In [36]:
# zoom value 9 to match https://minedbuildings.z5.web.core.windows.net/global-buildings/dataset-links.csv 
quad_keys = set()
for tile in list(mercantile.tiles(minx, miny, maxx, maxy, zooms=9)):
    quad_keys.add(mercantile.quadkey(tile))
quad_keys = list(quad_keys)
print(f"The input area spans {len(quad_keys)} tiles: {quad_keys}")

The input area spans 1 tiles: ['030232211']


In [37]:
idx = 0
combined_gdf = gpd.GeoDataFrame()
data_folder = '/Users/huajunchen/Library/Project/Python/segment-geo/segment_geospatial_api/notebook/global_buildings'

intersect_features = [] 

with tempfile.TemporaryDirectory() as tmpdir:
    for quad_key in tqdm(quad_keys):
        file_path = os.path.join(data_folder, f'{quad_key}_processed.json')

        if os.path.exists(file_path):
            gdf = gpd.read_file(file_path)       
            
            if not gdf.empty:
                intersecting = gdf[gdf.intersects(aoi_shape)]
                if not intersecting.empty:
                    for idx, feature in intersecting.iterrows():
                        intersect_features.append(feature)

print(f"\nTotal {len(intersect_features)} intersect features.")

# Save intersecting features to GeoJSON
if intersect_features:
    # Create GeoDataFrame from intersecting features
    intersect_gdf = gpd.GeoDataFrame(intersect_features, crs=4326)
    
    # Save to GeoJSON
    output_path = os.path.join(data_folder, 'intersecting_features.geojson')
    intersect_gdf.to_file(output_path, driver='GeoJSON')
    print(f"Intersecting features saved to: {output_path}")

100%|██████████| 1/1 [00:01<00:00,  1.88s/it]


Total 1 intersect features.
Intersecting features saved to: /Users/huajunchen/Library/Project/Python/segment-geo/segment_geospatial_api/notebook/global_buildings/intersecting_features.geojson
